## Neo4j Import of GraphRAG Result Parquet files
This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization.

### How does it work?
The notebook loads the parquet files from the output folder of your indexing process and loads them into Pandas dataframes. It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships.

All operations use `MERGE`, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement
```
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [2]:
GRAPHRAG_FOLDER="/home/yarikama/VGMARS/graph_rag_embedding_experimental/output/20240912-151541/artifacts"

In [3]:
import pandas as pd
from neo4j import GraphDatabase
import time
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
NEO4J_URI="bolt://localhost"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=os.environ["NEO4J_PASSWORD"]
NEO4J_DATABASE="neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [5]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

### Indexes and Constraints
Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect. Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [15]:
# create constraints, idempotent operation

constraint_statements = """
create constraint chunk_id          if not exists for (chunk:__Chunk__)                 require chunk.id            is unique;
create constraint document_id       if not exists for (document:__Document__)           require document.id         is unique;
create constraint community_id      if not exists for (community:__Community__)         require community.id        is unique;
create constraint entity_id         if not exists for (entity:__Entity__)               require entity.id           is unique;
create constraint entity_name       if not exists for (entity:__Entity__)               require entity.name         is unique;
create constraint covariate_title   if not exists for (covariate:__Covariate__)         require covariate.title     is unique;
create constraint relationship_id   if not exists for ()-[relationship:RELATED]->()     require relationship.id     is unique;
""".split(";")

for statement in constraint_statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id          if not exists for (chunk:__Chunk__)                 require chunk.id            is unique

create constraint document_id       if not exists for (document:__Document__)           require document.id         is unique

create constraint community_id      if not exists for (community:__Community__)         require community.id        is unique

create constraint entity_id         if not exists for (entity:__Entity__)               require entity.id           is unique

create constraint entity_name       if not exists for (entity:__Entity__)               require entity.name         is unique

create constraint covariate_title   if not exists for (covariate:__Covariate__)         require covariate.title     is unique

create constraint relationship_id   if not exists for ()-[relationship:RELATED]->()     require relationship.id     is unique


## Import Process
### Importing the Documents
We're loading the parquet file for the documents and create nodes with their ids and add the title property. We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [16]:
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet')
doc_df.head(2)

,id,text_unit_ids,raw_content,title,raw_content_embedding
0,474865e9db94180d0d99e5125ae04f2b,"[a2a6bc7fdeba51d448c87fd4b30ca1e8, 877021065a3...",Rise of the Guardians Script at IMSDb. The Int...,movie_Rise_of_the_Guardians.txt,"[0.016017707094181693, 0.02192370068985526, -0..."
1,dc56f172618f7849cb1cd4075359c4bb,"[fa119fc2d9dd1d9dacc117dfe72c5eb1, f7044ed1e87...",ïProject Gutenbergs The Adventure of the Cardb...,gutenberg_The_Adventure_of_the_Cardboard_Box.txt,"[0.03423983798018732, 0.030299222168653692, 0...."


In [20]:
# import documents
statement = """
// SET DOCUMENT AND ITS PROPERTIES
MERGE (document:__Document__ {id:value.id})
SET document += value {.title, .raw_content}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 11, 'nodes_created': 11, 'properties_set': 33}
11 rows in 0.12134313583374023 s.


11

In [ ]:
#test:
#MATCH (document: __Document__) RETURN document 

### Loading Text Units
We load the text units, create a node per id and set the text and number of tokens. Then we connect them to the documents that we created before.

In [21]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet')
text_df.head(2)

,id,text,text_embedding,n_tokens,document_ids,entity_ids,relationship_ids
0,057ff759b344cdcdea9924678f13e0b6,Yes Man Script at IMSDb. The Internet Movie Sc...,"[-0.04536692425608635, 0.08338487148284912, -0...",1024,[0015e9b933802296eb40df4f7a5cd5f3],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[af64651e237b43f2913082095772abd5, d98c4bed659..."
1,9ed083c1a3066c142cb7871b6fa6ded9,saw the ball drop and hugged all\n of you. Al...,"[-0.018350979313254356, 0.050461456179618835, ...",1024,[0015e9b933802296eb40df4f7a5cd5f3],"[4119fd06010c494caa07f439b333f4c5, 077d2820ae1...","[f14fb2a529644d9da08201b47a9c6277, 1a0d8ef2e5b..."


In [22]:
statement = """
// SET CHUNK AND ITS PROPERTIES
MERGE (chunk:__Chunk__ {id:value.id})
SET chunk += value {.text, .n_tokens}

// ADD RELATIONSHIPS BETWEEN CHUNKS AND DOCUMENTS
WITH chunk, value
UNWIND value.document_ids AS document_id
MATCH (document:__Document__ {id:document_id})
MERGE (chunk)-[:PART_OF]->(document)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 523, 'relationships_created': 523, 'nodes_created': 523, 'properties_set': 1569}
523 rows in 0.6427562236785889 s.


523

In [ ]:
#test:
#MATCH (chunk:__Chunk__) RETURN chunk

### Loading Nodes
For the nodes we store id, name, description, embedding (if available), human readable id.

In [23]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet')
entity_df.head(2)

,id,name,type,description,human_readable_id,graph_embedding,text_unit_ids,name_embedding,description_embedding
0,b45241d70f0e43fca764df95b2b81f77,CARL KENDALL,PERSON,Carl Kendall is a multifaceted character navig...,0,None,"[057ff759b344cdcdea9924678f13e0b6, 7464b81026d...","[0.010291694663465023, 0.022835588082671165, -...","[-0.010781682096421719, 0.029624300077557564, ..."
1,4119fd06010c494caa07f439b333f4c5,KATH,PERSON,Kath is a significant character in the narrati...,1,None,"[057ff759b344cdcdea9924678f13e0b6, 21c31cd17da...","[-0.006516867317259312, -0.0052177999168634415...","[-0.014255154877901077, 0.032462652772665024, ..."


In [24]:
entity_statement = """
// SET ENTITY AND ITS PROPERTIES
MERGE (entity:__Entity__ {id:value.id})
SET entity += value {.human_readable_id, .description, name:replace(value.name,'"','')}

// ADD VECTOR PROPERTY TO ENTITY
WITH entity, value
CALL db.create.setNodeVectorProperty(entity, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(entity, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node

// ADD RELATIONSHIPS BETWEEN CHUNKS AND ENTITIES
UNWIND value.text_unit_ids AS text_unit_id
MATCH (chunk:__Chunk__ {id:text_unit_id})
MERGE (chunk)-[:HAS_ENTITY]->(entity)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 2618, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 1940, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 1626, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 351, 'relationships_created': 499, 'nodes_created': 351, 'properties_set': 1404}
3351 rows in 7.6250505447387695 s.


3351

In [ ]:
# test:
# MATCH (entity:__Entity__) RETURN entity

### Import Relationships
For the relationships we find the source and target node by name, using the base `__Entity__` type. After creating the RELATED relationships, we set the description as attribute.

In [25]:
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet')
rel_df.head(2)

,source,target,weight,description,text_unit_ids,id,human_readable_id,description_embedding,source_degree,target_degree,rank
0,CARL KENDALL,KATH,9.0,Carl and Kath are in a relationship that is en...,[057ff759b344cdcdea9924678f13e0b6],af64651e237b43f2913082095772abd5,0,"[0.009233218617737293, 0.04426233470439911, -0...",16,7,23
1,CARL KENDALL,PETER,7.0,Peter is concerned about Carl's emotional stat...,[057ff759b344cdcdea9924678f13e0b6],d98c4bed659541d18ada07771a1121f1,1,"[-0.019778138026595116, -0.006448707543313503,...",16,9,25


In [26]:
rel_statement = """
// SET RELATIONSHIP AND ITS PROPERTIES
MATCH (source_entity:__Entity__ {name:replace(value.source,'"','')})
MATCH (target_entity:__Entity__ {name:replace(value.target,'"','')})
MERGE (source_entity)-[relationship:RELATED {id: value.id}]->(target_entity)
SET relationship += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}

// ADD VECTOR PROPERTY TO RELATIONSHIP
WITH relationship, value
CALL db.create.setRelationshipVectorProperty(relationship, "description_embedding", value.description_embedding)
RETURN count(*) as createdRelationships
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 566, 'properties_set': 3396}
4566 rows in 4.503098964691162 s.


4566

In [ ]:
# test:
# MATCH ()-[relationship:RELATED]->() RETURN relationship

### Importing Communities
For communities we import their id, title, level. We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [7]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet')
community_df.head(10)

,id,title,level,raw_community,relationship_ids,text_unit_ids
0,8,Community 8,0,8,"[af64651e237b43f2913082095772abd5, d98c4bed659...","[057ff759b344cdcdea9924678f13e0b6,7464b81026d1..."
1,2,Community 2,0,2,"[cb03bbaeee864b1f934b0f01cbb1442a, afe4a14c9e5...","[6dc1ceaf9bb2207acf2094d5fe63de4e,b05788939324..."
2,1,Community 1,0,1,"[d87f363868b74fa7b89e34577f85a699, 9ac8ab5457c...","[1523af964eec4b2efc1bf13ca451b51c,2b27af0b616e..."
3,14,Community 14,0,14,"[a112c85382a342b3b41ae622f87778cf, f4f0be99b99...","[116beefd8687bb5bf81490c60f416eaa,3901c66c5565..."
4,12,Community 12,0,12,"[4c85e0a6a28e476a85bb36c72c70f76f, 4eaed34b54c...","[043891482803e7772678b1d331638ad7,1523af964eec..."
5,18,Community 18,0,18,"[ff7ac62a97424812b0708448e22aa1c7, 4131754fd56...","[12447958acb218c77a8bfc92e7b8aa2e,345b3da65234..."
6,23,Community 23,0,23,"[b67546ee2f0b4781a3715dc2869acd95, 986e8261ffa...","[64c8bb6025d29c3ae070007121451e79,8526a2176ba9..."
7,3,Community 3,0,3,"[c2e20cd7de1a40319ab627da699d00fa, 3ab7b865103...","[70634860a4c5ee10e5fa35b2f1c7a845,786031626934..."
8,5,Community 5,0,5,"[41998fcd493c48b388046f02b212468a, a79a3dee10f...","[71b33f459a017bc5a2df4947d2ef6221,9a3ae72a1062..."
9,9,Community 9,0,9,"[956670a9c42349f3ac1ec78366ac23bc, f3575aa35e5...","[3837526d956baf19a20ad79a8221eb51,71b33f459a01..."


In [8]:
statement = """
// SET COMMUNITY AND ITS PROPERTIES
MERGE (community:__Community__ {id:value.id})
SET community += value {.level, .title}

// ADD RELATIONSHIPS BETWEEN CHUNKS AND COMMUNITIES
WITH *
UNWIND value.text_unit_ids as text_unit_id
MATCH (chunk:__Chunk__ {id:text_unit_id})
MERGE (community)-[:HAS_CHUNK]->(chunk)

// SET RELATIONSHIPS BETWEEN ENTITIES AND COMMUNITIES
WITH *
UNWIND value.relationship_ids as relationship_id
MATCH (source_entity:__Entity__)-[:RELATED {id:relationship_id}]->(target_entity:__Entity__)
MERGE (source_entity)-[:IN_COMMUNITY]->(community)
MERGE (target_entity)-[:IN_COMMUNITY]->(community)
RETURN count(distinct community) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 548, 'relationships_created': 15675, 'nodes_created': 548, 'properties_set': 1644}
548 rows in 0.3473327159881592 s.


548

In [ ]:
# test:
# MATCH (community:__Community__) RETURN community

### Importing Community Reports
Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about. For the findings we create the findings in context of the communities.

In [10]:
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet')
community_report_df.head(10)

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id,full_content_embedding,summary_embedding,title_embedding
0,534,"# Dwayne, Jay, and the Dynamics of Crime\n\nTh...",4,9.0,"Dwayne, Jay, and the Dynamics of Crime",The narrative provides deep insights into char...,The community centers around the characters Dw...,[{'explanation': 'Dwayne is portrayed as a mul...,"{\n ""title"": ""Dwayne, Jay, and the Dynamics...",cc180d9f-9abc-4459-938b-c70e3c62d877,"[0.03856884315609932, 0.06392262130975723, -0....","[0.019128641113638878, 0.05514286831021309, -0...","[0.041969262063503265, 0.04299576207995415, -0..."
1,535,# Donner-Wells National Bank and the Robbery N...,4,8.5,Donner-Wells National Bank and the Robbery Nar...,The narrative is highly significant as it expl...,The community centers around the Donner-Wells ...,[{'explanation': 'The Donner-Wells National Ba...,"{\n ""title"": ""Donner-Wells National Bank an...",51cf5ae2-e257-4253-bb90-69c5540dea08,"[0.010677372105419636, 0.021481269970536232, -...","[-0.0006910123047418892, 0.012934575788676739,...","[0.019868899136781693, 0.004599020350724459, -..."
2,536,# Caterina and Pasqualino: Love Amidst War\n\n...,4,9.0,Caterina and Pasqualino: Love Amidst War,The narrative is highly significant as it expl...,The community centers around the characters Ca...,[{'explanation': 'Caterina represents the emot...,"{\n ""title"": ""Caterina and Pasqualino: Love...",43d79808-0fd1-402d-9c1e-3313a0ac8363,"[0.044687822461128235, -0.004155505448579788, ...","[0.024521034210920334, -0.00991173554211855, 0...","[-0.002926247427240014, -0.005339326336979866,..."
3,537,# Pasqualino and the Dynamics of Community\n\n...,4,9.0,Pasqualino and the Dynamics of Community,The narrative provides deep insights into char...,"The community centers around Pasqualino, a you...",[{'explanation': 'Pasqualino's character is pi...,"{\n ""title"": ""Pasqualino and the Dynamics o...",b7961940-b66b-4c7d-a9b0-9ebf926265df,"[0.03973129764199257, 0.03397233784198761, 0.0...","[0.014332879334688187, 0.019663406535983086, -...","[0.009135249070823193, 0.014460430480539799, 0..."
4,538,"# Clay, Aisha, and the Dynamics of Conflict\n\...",4,9.0,"Clay, Aisha, and the Dynamics of Conflict",The narrative is highly significant as it intr...,This community centers around the complex rela...,[{'explanation': 'The bathroom serves as a mul...,"{\n ""title"": ""Clay, Aisha, and the Dynamics...",c8f0e5ff-0025-4f06-a5e5-bb70b5dcec02,"[0.041022852063179016, 0.05283648893237114, 0....","[0.047144606709480286, 0.054332319647073746, 0...","[0.0344376303255558, 0.031317442655563354, 0.0..."
5,539,# Goliath Terminal and Covert Operations\n\nTh...,4,8.5,Goliath Terminal and Covert Operations,The rating reflects the significant role of th...,The community centers around the Goliath Termi...,[{'explanation': 'The Goliath Terminal serves ...,"{\n ""title"": ""Goliath Terminal and Covert O...",8a1079c8-4dad-48e4-92bd-b8194ff527e3,"[0.024007773026823997, 0.06874759495258331, 0....","[0.045322585850954056, 0.0635654479265213, 0.0...","[0.02044178731739521, 0.025230377912521362, 0...."
6,541,# Footsteps and the Watcher in Narrative Tensi...,4,8.5,Footsteps and the Watcher in Narrative Tension,The narrative's use of auditory motifs and uns...,The community centers around the auditory moti...,[{'explanation': 'The entity 'Footsteps' serve...,"{\n ""title"": ""Footsteps and the Watcher in ...",a16b4975-51ad-42c8-b618-8057f0ad0a3e,"[0.026668185368180275, 0.04140794277191162, -0...","[0.016440140083432198, 0.049465395510196686, -...","[0.018672898411750793, 0.050637371838092804, -..."
7,542,# Emilia: The Heart of Rivalry and Desire\n\nT...,4,9.5,Emilia: The Heart of Rivalry and Desire,The narrative's depth in exploring emotional c...,"The community centers around Emilia, a complex...",[{'explanation': 'Emilia is portrayed as the o...,"{\n ""title"": ""Emilia: The Heart of Rivalry ...",ade0522b-8941-402c-8c28-12aa23a0c

In [11]:
# import communities
community_statement = """
// SET COMMUNITY REPORT AND ITS PROPERTIES
MERGE (community:__Community__ {id:value.community})
SET community += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}

// ADD VECTOR PROPERTY TO COMMUNITY REPORT
WITH community, value
CALL db.create.setNodeVectorProperty(community, "summary_embedding", value.summary_embedding)

// ADD RELATIONSHIPS BETWEEN COMMUNITIES AND FINDINGS
WITH community, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH community, value, finding_idx, value.findings[finding_idx] as value_finding
MERGE (community)-[:HAS_FINDING]->(finding:Finding {id:finding_idx})
SET finding += value_finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 2971, 'relationships_created': 2971, 'nodes_created': 2971, 'properties_set': 12201}
548 rows in 1.7159245014190674 s.


548

### Importing Covariates
Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

**By default, covariates are not included in the output, so the file might not exists in your output if you didn't set the configuration to extract claims**

In [18]:
"""
# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')
# cov_df.head(2)
"""

"\n# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')\n# cov_df.head(2)\n"

In [19]:
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
# batched_import(cov_statement, cov_df)